In [1]:
import cv2
import pickle
import numpy as np

# Load video and parking positions
cap = cv2.VideoCapture("carPark.mp4")
with open("car_park_pos", "rb") as f:
    pos_list = pickle.load(f)

# Define parking space dimensions
w, h = 107, 48

def check_parking_space(img_free):
    global color, thickness
    spaces, totals, filled = 0, 0, 0

    for pos in pos_list:
        x, y = pos
        totals += 1

        # Extract the region of interest (ROI) for each parking space
        roi = img_free[y:y + h, x:x + w]

        # Count non-zero pixels in the ROI
        count = cv2.countNonZero(roi)

        # Display the count on the image
        cv2.putText(img, str(count), (x, y + h - 3), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

        # Check if the parking space is filled
        if count > 1000:
            color, thickness = (0, 255, 0), 5
            filled += 1
        else:
            color, thickness = (0, 0, 255), 3
            spaces += 1

        # Draw rectangles around parking spaces
        cv2.rectangle(img, pos, (x + w, y + h), color, thickness)

    # Display the total empty spaces at the bottom of the image
    cv2.rectangle(img, (40, 20), (480, 60), (0, 0, 0), -1)
    cv2.putText(img, f"Total Empty Space: {spaces}/{totals}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (255, 255, 255), 2, cv2.LINE_AA)

while True:
    # Read a frame from the video
    success, img = cap.read()

    # Convert the frame to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to reduce noise
    img_blur = cv2.GaussianBlur(img_gray, (3, 3), 1)

    # Apply Adaptive Thresholding
    img_threshold = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 25, 16)

    # Apply Median Blur
    img_median = cv2.medianBlur(img_threshold, 5)

    # Dilate the image
    kernel = np.ones((3, 3), np.uint8)
    img_dilate = cv2.dilate(img_median, kernel, iterations=1)

    # Check parking spaces
    check_parking_space(img_dilate)

    # Display the image
    cv2.imshow('Car Parking Lot', img)

    # Break the loop when 'Esc' key is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
